In [ ]:
import numpy as np
import cvxpy as cp

from problems.data_generation import generate_problem_data
from problems.problem_definition import BilevelProblem
from algorithms.barrier_blo_box import BarrierBLO
from algorithms.blocc import BLOCC
from algorithms.implicit_gradient_descent import IGD
from algorithms.BiC_GAFFA import BiC_GAFFA

n = 60
seed = 9

data = generate_problem_data(n, seed)
problem = BilevelProblem(data)

x_init = np.zeros(n)
y_init = np.zeros(n)
z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
theta_init = np.zeros(n)
y_g_init = np.zeros(n)
y_F_init = np.zeros(n)
mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
hparams = {
        'barrier_blo': {
            'M': 0.001,
            't': 0.01,
            'alpha_x': 0.002,
            'alpha_y': 0.1,
            'beta_y': 1,
            'epsilon_x': 0.1,
            'epsilon_y': 0.01,
            'inner_max_iters': 1000000,
            'outer_max_iters': 100000
        },
        'blocc': {
            'gamma': 100,
            'alpha_x': 0.005,
            'alpha_g_y': 0.01,
            'alpha_F_y': 0.0002,
            'beta_g_y': 0.01,
            'beta_F_y': 0.01,
            'epsilon_x': 0.1,
            'epsilon_inner_y_g': 0.01,
            'epsilon_outer_y_g': 0.01,
            'epsilon_inner_y_F': 0.01,
            'epsilon_outer_y_F': 0.01,
            'maxmin_g_outer_max_iters': 5000,
            'maxmin_F_outer_max_iters': 5000,
            'maxmin_g_inner_max_iters': 5000,
            'maxmin_F_inner_max_iters': 5000,
            'main_max_iters': 5000 
        },
        'bic_gaffa':{
            'alpha':0.01,
            'c':1,
            'tau':1.3,
            'gamma_1':10,
            'gamma_2':1,
            'eta':1,
            'r':1,
            'epsilon':0.1,
            'max_iters':100
        },
        'IGD': {
            'M': 1e-3,
            't': 1e-3,
            'alpha_x': 0.001,
            'alpha_y': 0.001,
            'epsilon_x': 1e-3,
            'epsilon_y': 1e-3,
            'inner_max_iters': 1000,
            'outer_max_iters': 1000
        }
    }

# BiC_GAFFA Algorithm

In [2]:
bic_algo = BiC_GAFFA(problem, hparams)

x_opt_bic, y_opt_bic, history = bic_algo.bic_gaffa(x_init, y_init, z_init, theta_init)
print(problem.f(x_opt_bic, y_opt_bic))

f(x,y)=-25.160383029879597,grad_norm = 79.43628365149956
f(x,y)=-26.274633437488433,grad_norm = 15.167974687761916
f(x,y)=-26.29272344516163,grad_norm = 1.7689325776616516
f(x,y)=-26.29277502778561,grad_norm = 0.0866473914863969
f(x,y)=-26.29277509797273,grad_norm = 0.002891309514752457
f(x,y)=-26.292775099565418,grad_norm = 0.00039885293459707876
f(x,y)=-26.292775099658954,grad_norm = 8.93297876324002e-05
f(x,y)=-26.292775099668383,grad_norm = 2.6386357885821548e-05
f(x,y)=-26.29277509966976,grad_norm = 9.405170609565252e-06
f(x,y)=-26.292775099670028,grad_norm = 3.854748229731183e-06
f(x,y)=-26.292775099670088,grad_norm = 1.7625493921787005e-06
f(x,y)=-26.292775099670102,grad_norm = 8.80630517946008e-07
f(x,y)=-26.292775099670113,grad_norm = 4.735621622798985e-07
f(x,y)=-26.29277509967011,grad_norm = 2.7094320608028766e-07
f(x,y)=-26.292775099670102,grad_norm = 1.6343842285428162e-07
f(x,y)=-26.292775099670123,grad_norm = 1.031877088654911e-07
f(x,y)=-26.292775099670116,grad_norm = 6

# BiC_GAFFA Feasibility Check

In [3]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_bic, y_opt_bic, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_bic, y_opt_bic, i) for i in range(problem.num_constraints_h2)]

y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_bic, y_opt_bic)
print(f"f_theoretical_opt_result={problem.f(x_opt_bic, y_original_opt_blocc)}, f_blocc_numerical_result={problem.f(x_opt_bic, y_opt_bic)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_bic, y_original_opt_blocc)}, g_blocc_numerical_result={problem.g(x_opt_bic, y_opt_bic)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=56.96459582870526, f_blocc_numerical_result=-26.29277509967011
g_theoretical_opt_result=-48.53645524012808, g_blocc_numerical_result=0.0
h_1[0] = -0.22763474112308701
h_1[1] = -0.8779064257387715
h_1[2] = -0.5339547027139351
h_1[3] = -0.35116317302613753
h_1[4] = -0.2871273405685729
h_1[5] = -0.02584656083667536
h_1[6] = -0.8602919490571069
h_1[7] = -0.6838406791631587
h_1[8] = -0.2892123596437677
h_1[9] = -0.9657110970341924
h_1[10] = -0.1722560534396942
h_1[11] = -0.3077636987611079
h_1[12] = -0.37786382086100334
h_1[13] = -0.8831616191677
h_1[14] = -0.6853745312971752
h_1[15] = -0.3535801796315752
h_1[16] = -0.4083224434398852
h_1[17] = -0.35555446061126583
h_1[18] = -0.8660479913457553
h_1[19] = -0.043299747458223736
h_2[0] = -1.0
h_2[1] = -1.0
h_2[2] = -1.0
h_2[3] = -1.0
h_2[4] = -1.0
h_2[5] = -1.0
h_2[6] = -1.0
h_2[7] = -1.0
h_2[8] = -1.0
h_2[9] = -1.0
h_2[10] = -1.0
h_2[11] = -1.0
h_2[12] = -1.0
h_2[13] = -1.0
h_2[14] = -1.0
h_2[15] = -1.0
h_2[16] = -1.0

# BLOCC Algorithm

In [127]:
blocc_algo = BLOCC(problem, hparams)


x_opt_blocc, y_opt_blocc, history = blocc_algo.blocc(x_init, y_g_init, y_F_init, mu_g_init, mu_F_init)
print(problem.f(x_opt_blocc, y_opt_blocc))



Main iteration 1
Inner iter for L_g=0, Projected Gradient norm w.r.t y of L_g=77.81704153080214
Inner iter for L_g=1, Projected Gradient norm w.r.t y of L_g=31.767444706447122
Inner iter for L_g=2, Projected Gradient norm w.r.t y of L_g=13.466603359198848
Inner iter for L_g=3, Projected Gradient norm w.r.t y of L_g=5.877338960259464
Inner iter for L_g=4, Projected Gradient norm w.r.t y of L_g=2.6228919389201626
Inner iter for L_g=5, Projected Gradient norm w.r.t y of L_g=1.1907891827373167
Inner iter for L_g=6, Projected Gradient norm w.r.t y of L_g=0.547899178382287
Inner iter for L_g=7, Projected Gradient norm w.r.t y of L_g=0.2547703192424772
Inner iter for L_g=8, Projected Gradient norm w.r.t y of L_g=0.1194667142495143
Inner iter for L_g=9, Projected Gradient norm w.r.t y of L_g=0.05640002810274485
Inner iter for L_g=10, Projected Gradient norm w.r.t y of L_g=0.02677275502985416
Inner iter for L_g=11, Projected Gradient norm w.r.t y of L_g=0.012766121044104642
Inner loop for L_g c

# BLOCC Feasibility Check

In [128]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]

y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_blocc, y_opt_blocc)
print(f"f_theoretical_opt_result={problem.f(x_opt_blocc, y_original_opt_blocc)}, f_blocc_numerical_result={problem.f(x_opt_blocc, y_opt_blocc)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_blocc, y_original_opt_blocc)}, g_blocc_numerical_result={problem.g(x_opt_blocc, y_opt_blocc)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=55.7499439284052, f_blocc_numerical_result=52.00837229744439
g_theoretical_opt_result=-47.29700898498122, g_blocc_numerical_result=-47.2784833279084
h_1[0] = -1.278292590992046
h_1[1] = -0.4283731804923677
h_1[2] = 2.220446049250313e-16
h_1[3] = 5.551115123125783e-17
h_1[4] = -0.47419381499427393
h_1[5] = -0.05873411556606156
h_1[6] = -0.6585673170732038
h_1[7] = -3.3306690738754696e-16
h_1[8] = -0.9992363969505241
h_1[9] = -1.0577634966243086
h_1[10] = -8.326672684688674e-17
h_1[11] = -1.3825592388539134
h_1[12] = -0.33384351151720015
h_1[13] = -1.4748649660857212
h_1[14] = -1.4511374636608712
h_1[15] = -0.06445563202772092
h_1[16] = -0.3359472617496754
h_1[17] = 1.1102230246251565e-16
h_1[18] = -0.589338760535262
h_1[19] = -5.551115123125783e-17
h_2[0] = -1.076720586797243
h_2[1] = -1.020143996480367
h_2[2] = -0.9712837176051434
h_2[3] = -1.0594659465016099
h_2[4] = -1.0871526826375233
h_2[5] = -0.8247985459414764
h_2[6] = -1.021433200508073
h_2[7] = -1.15993

# Implicit Gradient Descent Algorithm

In [49]:
IGD_algo = IGD(problem, hparams)

x_opt_IGD, y_opt_IGD, history = IGD_algo.upper_loop(x_init, y_init)

print("Optimized x:", x_opt_IGD)
print("Optimized y:", y_opt_IGD)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 200
f(x, y) = 95.45037851360806, grad_norm = 76.59673418115331
Outer iteration 2
Inner loop converged at iteration 112
f(x, y) = 91.19500851740221, grad_norm = 67.2257525452191
Outer iteration 3
Inner loop converged at iteration 110
f(x, y) = 87.9160198480594, grad_norm = 59.00630049516197
Outer iteration 4
Inner loop converged at iteration 107
f(x, y) = 85.39015297241798, grad_norm = 51.796437571852486
Outer iteration 5
Inner loop converged at iteration 105
f(x, y) = 83.44417540097366, grad_norm = 45.471557030758525
Outer iteration 6
Inner loop converged at iteration 102
f(x, y) = 81.94475905747844, grad_norm = 39.922553604489096
Outer iteration 7
Inner loop converged at iteration 100
f(x, y) = 80.7892951506114, grad_norm = 35.053784045327106
Outer iteration 8
Inner loop converged at iteration 98
f(x, y) = 79.8987827656407, grad_norm = 30.781487703596458
Outer iteration 9
Inner loop converged at iteration 95
f(x, y) = 79.212391899265

# Implicit Gradient Descent Feasibility Check

In [50]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h2)]

y_original_opt_IGD = barrier_algo.Interior_inner_loop(x_opt_IGD, y_opt_IGD)
print(f"f_theoretical_opt_result={problem.f(x_opt_IGD, y_original_opt_IGD)}, f_IGD_numerical_result={problem.f(x_opt_IGD, y_opt_IGD)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_IGD, y_original_opt_IGD)}, g_IGD_numerical_result={problem.g(x_opt_IGD, y_opt_IGD)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=59.04726374415966, f_IGD_numerical_result=76.90380080947456
g_theoretical_opt_result=-47.905074255466786, g_IGD_numerical_result=-51.811093935401786
h_1[0] = -1.818606806017621
h_1[1] = -0.6332653285456722
h_1[2] = 0.692060467876725
h_1[3] = -0.19201660481909777
h_1[4] = -0.600298040133991
h_1[5] = -0.2737396315591377
h_1[6] = -1.012366935480417
h_1[7] = 0.4450296606186358
h_1[8] = -1.4287421110645298
h_1[9] = -0.8173446049192401
h_1[10] = 1.0874902560609827
h_1[11] = -1.4789022155643556
h_1[12] = 0.10827124470942368
h_1[13] = -1.786263928772887
h_1[14] = -0.8903993418731315
h_1[15] = -0.07504978497512865
h_1[16] = -0.3015547409733911
h_1[17] = -0.21462543837571568
h_1[18] = -0.693290361744195
h_1[19] = 0.5416734352752851
h_2[0] = -1.0929089554460576
h_2[1] = -0.9654870816489937
h_2[2] = -1.0237375559765334
h_2[3] = -1.0192583555543342
h_2[4] = -1.044948936664087
h_2[5] = -0.877464459734365
h_2[6] = -1.0860014094637798
h_2[7] = -1.2018995373962
h_2[8] = -1.4768

# BFBM Algorithm

In [96]:
barrier_algo = BarrierBLO(problem, hparams)

# y_proj = barrier_algo.project_to_constraints(x_init, y_init)

# h = problem.h_1(x_init, y_proj, 0)

# print(y_proj)

# print(h)



x_opt_barrier, y_opt_barrier, history = barrier_algo.upper_loop(x_init, y_init)

print("Optimized x:", x_opt_barrier)
print("Optimized y:", y_opt_barrier)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 84
f(x, y) = 77.1715615837633, grad_norm of hyperfunction= 87.45817064272863
Outer iteration 2
Inner loop converged at iteration 74
f(x, y) = 78.90024953534076, grad_norm of hyperfunction= 392.8141925153819
Outer iteration 3
Inner loop converged at iteration 90
f(x, y) = 29.23452608787925, grad_norm of hyperfunction= 150.63346563071727
Outer iteration 4
Inner loop converged at iteration 85
f(x, y) = 13.444898881685411, grad_norm of hyperfunction= 102.62089018132221
Outer iteration 5
Inner loop converged at iteration 84
f(x, y) = 7.4682148832218616, grad_norm of hyperfunction= 76.75084902098465
Outer iteration 6
Inner loop converged at iteration 80
f(x, y) = 3.8799643720035704, grad_norm of hyperfunction= 64.7094077398494
Outer iteration 7
Inner loop converged at iteration 80
f(x, y) = 1.3128754379971141, grad_norm of hyperfunction= 55.90741593897381
Outer iteration 8
Inner loop converged at iteration 80
f(x, y) = -0.6132860446946751, 

KeyboardInterrupt: 

In [23]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]

y_original_opt_bfbm = barrier_algo.Interior_inner_loop(x_opt_barrier, y_opt_barrier)
print(f"f_theoretical_opt_result={problem.f(x_opt_barrier, y_original_opt_bfbm)}, f_bfbm_numerical_result={problem.f(x_opt_barrier, y_opt_barrier)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_barrier, y_original_opt_bfbm)}, g_bfbm_numerical_result={problem.g(x_opt_barrier, y_opt_barrier)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=-15.306563751288039, f_bfbm_numerical_result=-15.153427501682827
g_theoretical_opt_result=-24.423517710938498, g_bfbm_numerical_result=-24.333709631271457
h_1[0] = -0.8530995669552834
h_1[1] = -0.2613523576031589
h_1[2] = -0.001278392080932278
h_1[3] = -0.07502091218352613
h_1[4] = -0.375177649114236
h_1[5] = -0.049867996945927556
h_1[6] = -0.8780341222931158
h_1[7] = -0.0014893580201749135
h_1[8] = -0.5236765700613755
h_1[9] = -0.6984091586204063
h_1[10] = -0.00157056475541284
h_1[11] = -0.44529108173842336
h_1[12] = -0.013355775506502299
h_1[13] = -1.1743797753157472
h_1[14] = -0.760361498677131
h_1[15] = -0.00692658288940684
h_1[16] = -1.1122354621821224
h_1[17] = -0.0022449890145965767
h_1[18] = -0.003619708732381821
h_1[19] = -0.0012898782281610122
h_2[0] = -1.0393108199461274
h_2[1] = -1.0373869892836591
h_2[2] = -0.9414777455098461
h_2[3] = -1.063995152673622
h_2[4] = -1.0546833198897954
h_2[5] = -0.9322605401319755
h_2[6] = -0.9065338507107821
h_2[7] = 

# Plot

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(times, grad_norms, marker='o', markersize=1)
plt.xlabel('CPU Time (s)')
plt.ylabel('Gradient Norm')
plt.title('Gradient Norm vs CPU Time')
plt.yscale('log')
plt.grid(True)
plt.show()
